In [1]:
# ======================================
# 🔹 Install dependencies
# ======================================
!pip install yfinance PyPortfolioOpt torch torchvision torchaudio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 6.1 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [3]:
# ======================================
# 🔹 Load Multi-Asset / Macro Data
# ======================================
def safe_load(symbol, start="2018-01-01"):
    data = yf.download(symbol, start=start)
    if isinstance(data, pd.DataFrame):
        close_col = data.columns[-1]  # ใช้คอลัมน์สุดท้าย (yfinance auto-adjust)
        return data[close_col].rename(symbol)
    return None

assets = ["BTC-USD", "^GSPC", "GC=F", "^IXIC"]  # Bitcoin, S&P500, Gold, NASDAQ
dfs = [safe_load(sym) for sym in assets]
df = pd.concat(dfs, axis=1).dropna()
print("✅ Data loaded:", df.shape)

/tmp/ipython-input-3788204494.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3788204494.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3788204494.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3788204494.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start)
[*********************100%***********************]  1 of 1 completed

✅ Data loaded: (1969, 4)


In [4]:
# ======================================
# 3) Feature engineering (Multi-asset + Regime)
# ======================================
import numpy as np
import pandas as pd

# H = horizon วันล่วงหน้า (แก้เป็น 30 ได้)
H = 3
base = df.copy().astype(float)     # df มาจากส่วนโหลดข้อมูลของคุณ

def rsi_14(x: pd.Series):
    up = x.diff().clip(lower=0).rolling(14, min_periods=14).mean()
    dn = x.diff().clip(upper=0).abs().rolling(14, min_periods=14).mean()
    return 100 - (100/(1 + (up/dn)))

new_features = {}
for c in base.columns:
    price = base[c].replace(0, np.nan)

    with np.errstate(divide="ignore", invalid="ignore"):
        lr = np.log(price/price.shift(1))
    new_features[f"{c}_logret"] = lr.replace([np.inf, -np.inf], np.nan)

    new_features[f"{c}_vol"] = price.pct_change().rolling(14, min_periods=14).std()
    new_features[f"{c}_rsi"] = rsi_14(price)

feat_df = pd.DataFrame(new_features, index=base.index)

# ----- BTC-specific -----
btc = base["BTC-USD"]
feat_df["BTC-USD_ma7"]      = btc.rolling(7,  min_periods=7).mean()
feat_df["BTC-USD_ma21"]     = btc.rolling(21, min_periods=21).mean()
feat_df["BTC-USD_momentum"] = btc / btc.shift(10) - 1
macd_fast = btc.ewm(12, adjust=False).mean()
macd_slow = btc.ewm(26, adjust=False).mean()
macd = macd_fast - macd_slow
feat_df["BTC-USD_macd"]   = macd
feat_df["BTC-USD_signal"] = macd.ewm(9, adjust=False).mean()

# ----- Cross-asset correlations (14D) -----
if all(x in base.columns for x in ["BTC-USD","^GSPC","GC=F","^IXIC"]):
    feat_df["SP500_corr"]  = base["BTC-USD"].rolling(14, min_periods=14).corr(base["^GSPC"])
    feat_df["GOLD_corr"]   = base["BTC-USD"].rolling(14, min_periods=14).corr(base["GC=F"])
    feat_df["NASDAQ_corr"] = base["BTC-USD"].rolling(14, min_periods=14).corr(base["^IXIC"])

# ----- Regime features: 30D realized vol ของ BTC -----
btc_vol30 = btc.pct_change().rolling(30, min_periods=30).std()
feat_df["BTC_vol30"] = btc_vol30
q1, q2 = btc_vol30.quantile([0.33, 0.66])
feat_df["regime_low"]  = (btc_vol30 <= q1).astype(float)
feat_df["regime_mid"]  = ((btc_vol30 > q1) & (btc_vol30 <= q2)).astype(float)
feat_df["regime_high"] = (btc_vol30 > q2).astype(float)

# ----- รวม + target -----
data = pd.concat([base, feat_df], axis=1)
data["target"] = (data["BTC-USD"].shift(-H) > data["BTC-USD"]).astype(int)

# clean
data = data.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)
print("✅ Feature table:", data.shape)


/tmp/ipython-input-3449916581.py:24: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  new_features[f"{c}_vol"] = price.pct_change().rolling(14, min_periods=14).std()
/tmp/ipython-input-3449916581.py:24: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  new_features[f"{c}_vol"] = price.pct_change().rolling(14, min_periods=14).std()


✅ Feature table: (1660, 29)


In [5]:
# ======================================
# 4) Prepare dataset (split, scale, sequence)
# ======================================
from sklearn.model_selection import train_test_split

# เลือกฟีเจอร์ที่สร้างไว้ใน Step 3
feature_keys = ["logret","_vol","_rsi","ma","momentum","macd","signal","_corr","regime_","BTC_vol30"]
features = [c for c in data.columns if any(k in c for k in feature_keys)]

assert "target" in data.columns, "ไม่พบคอลัมน์ target จากขั้นที่ 3"
assert len(features) > 0, "ไม่พบฟีเจอร์จากขั้นที่ 3"

# แยก train/val ตามเวลา (ไม่สลับลำดับ)
split = int(len(data)*0.8)
train_df = data.iloc[:split].copy()
val_df   = data.iloc[split:].copy()

# scale เฉพาะบน train แล้วค่อย transform val เพื่อกัน data leakage
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
val_df[features]   = scaler.transform(val_df[features])

# ทำเป็นลำดับเวลา
class SeqDataset(Dataset):
    def __init__(self, frame, fcols, seq_len=128):
        X, y = [], []
        vals = frame[fcols].values
        tars = frame["target"].values.astype(np.float32)
        for i in range(len(frame) - seq_len):
            X.append(vals[i:i+seq_len])
            y.append(tars[i+seq_len])
        self.X = torch.tensor(np.array(X), dtype=torch.float32)
        self.y = torch.tensor(np.array(y), dtype=torch.float32).unsqueeze(1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

SEQ_LEN = 128
BATCH   = 64
train_loader = DataLoader(SeqDataset(train_df, features, SEQ_LEN), batch_size=BATCH, shuffle=True)
val_loader   = DataLoader(SeqDataset(val_df,   features, SEQ_LEN), batch_size=BATCH, shuffle=False)

print(f"train_size={len(train_df)}  val_size={len(val_df)}  n_features={len(features)}")


train_size=1328  val_size=332  n_features=24


In [6]:
# ======================================
# 5) GRU model (regularized)
# ======================================
class GRUModel(nn.Module):
    def __init__(self, in_dim, hidden=256, layers=3, dropout=0.5):
        super().__init__()
        self.gru = nn.GRU(
            input_size=in_dim,
            hidden_size=hidden,
            num_layers=layers,
            batch_first=True,
            dropout=dropout if layers > 1 else 0.0
        )
        self.drop = nn.Dropout(dropout)
        self.out  = nn.Linear(hidden, 1)  # binary logit
    def forward(self, x):
        h, _ = self.gru(x)           # (B, T, H)
        h_last = h[:, -1, :]         # (B, H)
        h_last = self.drop(h_last)
        return self.out(h_last)      # (B, 1)

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = GRUModel(in_dim=len(features), hidden=256, layers=3, dropout=0.5).to(device)


In [7]:
# ======================================
# 6) Train / Evaluate utils (Early stopping + Cosine LR)
# ======================================
def eval_model(model, loader, threshold=0.5):
    model.eval()
    y_true, y_prob, tot = [], [], 0.0
    loss_fn = nn.BCEWithLogitsLoss()
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logit = model(xb)
            loss  = loss_fn(logit, yb)
            tot  += loss.item()
            prob  = torch.sigmoid(logit).cpu().numpy().ravel()
            y_prob.extend(prob)
            y_true.extend(yb.cpu().numpy().ravel())
    y_true = np.array(y_true); y_prob = np.array(y_prob)
    y_pred = (y_prob > threshold).astype(int)
    # กันเคส all-one/all-zero
    if len(np.unique(y_true)) < 2:
        f1, auc, acc = 0.0, 0.5, (y_pred==y_true).mean()
    else:
        f1  = f1_score(y_true, y_pred, zero_division=0)
        auc = roc_auc_score(y_true, y_prob)
        acc = (y_pred==y_true).mean()
    return tot/len(loader), f1, auc, acc, y_true, y_prob

def train_model(model, train_loader, val_loader, epochs=200, lr=2e-4, patience=10):
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=max(20, epochs//2), eta_min=1e-6)
    loss_fn = nn.BCEWithLogitsLoss()

    best_val = np.inf
    patience_ctr = 0
    history = {"train_loss": [], "val_loss": [], "f1": [], "auc": [], "acc": []}

    for ep in range(1, epochs+1):
        model.train()
        total = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            logit  = model(xb)
            loss   = loss_fn(logit, yb)
            opt.zero_grad(); loss.backward(); opt.step()
            total += loss.item()
        scheduler.step()

        val_loss, f1, auc, acc, _, _ = eval_model(model, val_loader, threshold=0.5)
        history["train_loss"].append(total/len(train_loader))
        history["val_loss"].append(val_loss)
        history["f1"].append(f1)
        history["auc"].append(auc)
        history["acc"].append(acc)

        print(f"Epoch {ep:03d}: train={total/len(train_loader):.4f}  val={val_loss:.4f}  F1={f1:.3f}  AUC={auc:.3f}  ACC={acc:.3f}")

        # early stopping ตัดสินด้วย val_loss
        if val_loss < best_val - 1e-4:
            best_val = val_loss
            patience_ctr = 0
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            patience_ctr += 1
            if patience_ctr >= patience:
                print(f"⏹️ Early stopping at epoch {ep}")
                break

    if 'best_state' in locals():
        model.load_state_dict(best_state)
    return model, history


In [8]:
print("🔹 Total features selected:", len(features))
print(features[:10], "...")


🔹 Total features selected: 24
['BTC-USD_logret', 'BTC-USD_vol', 'BTC-USD_rsi', '^GSPC_logret', '^GSPC_vol', '^GSPC_rsi', 'GC=F_logret', 'GC=F_vol', 'GC=F_rsi', '^IXIC_logret'] ...


In [ ]:
# ======================================
# 7) Train
# ======================================
model, history = train_model(model, train_loader, val_loader, epochs=200, lr=2e-4, patience=10)


In [ ]:
# ======================================
# 8) Pick threshold by F1 on validation
# ======================================
_, _, _, _, y_true_val, y_prob_val = eval_model(model, val_loader, threshold=0.5)

ths = np.linspace(0.2, 0.8, 25)
f1s, accs = [], []
for t in ths:
    pred = (y_prob_val > t).astype(int)
    f1s.append(f1_score(y_true_val, pred, zero_division=0))
    accs.append((pred == y_true_val).mean())

best_idx = int(np.argmax(f1s))
best_th  = float(ths[best_idx])
print(f"Best threshold (F1 on val) = {best_th:.3f}  F1={f1s[best_idx]:.3f}  ACC={accs[best_idx]:.3f}")


In [ ]:
# ======================================
# 9) Final validation report + plots (ROC / PR / Loss curves)
# ======================================
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix

# loss curves
plt.figure(figsize=(8,4))
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"],   label="Val Loss")
plt.legend(); plt.title("Loss Curves"); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.show()

# ROC
fpr, tpr, _ = roc_curve(y_true_val, y_prob_val)
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"AUC={roc_auc_score(y_true_val, y_prob_val):.3f}")
plt.plot([0,1],[0,1],'--')
plt.legend(); plt.title("ROC (Validation)"); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.show()

# PR
prec, rec, _ = precision_recall_curve(y_true_val, y_prob_val)
plt.figure(figsize=(6,6))
plt.plot(rec, prec)
plt.title("Precision-Recall (Validation)"); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.show()

# Confusion matrix at best_th
y_pred_best = (y_prob_val > best_th).astype(int)
cm = confusion_matrix(y_true_val, y_pred_best)
print("Confusion matrix @ best_th:\n", cm)

# สรุปตัวเลขสุดท้าย @ best_th
final_f1  = f1_score(y_true_val, y_pred_best, zero_division=0)
final_acc = (y_pred_best == y_true_val).mean()
final_auc = roc_auc_score(y_true_val, y_prob_val)
print(f"Final (val)  AUC={final_auc:.3f}  F1@best_th={final_f1:.3f}  ACC@best_th={final_acc:.3f}")


In [ ]:
# ======================================
# 10) Sanity checks: class balance และ distribution ของ prob
# ======================================
pos_rate = y_true_val.mean()
print(f"Positive ratio (val) = {pos_rate:.3f}")

plt.figure(figsize=(6,4))
plt.hist(y_prob_val, bins=30)
plt.title("Predicted probability distribution (val)")
plt.xlabel("P(y=1)"); plt.ylabel("count"); plt.show()
